<a href="https://colab.research.google.com/github/colaquafina/Roller-King/blob/main/decoding_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
sns.set_context('poster')

from glob import glob

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedShuffleSplit,cross_validate

In [2]:
fname = "kay_labels.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/r638s/download
fname = "kay_labels_val.npy"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/yqb3e/download
fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

with np.load(fname) as dobj:
    dat = dict(**dobj)
labels = np.load('kay_labels.npy')
val_labels = np.load('kay_labels_val.npy')

if not os.path.exists('Roller-King'):
    !git clone https://github.com/colaquafina/Roller-King.git

Cloning into 'Roller-King'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.


In [8]:
for item in glob('Roller-King/judge_*.npy'):
    print(item.split('/')[-1],np.load(item).shape)

judge_1.npy (440,)
judge_2.npy (442,)
judge_3.npy (440,)
judge_4.npy (430,)


In [3]:
manual_labels = np.concatenate([np.load(item) for item in glob('Roller-King/judge_*.npy')])

In [4]:
unique_roi_index = np.unique(dat['roi'])
unique_roi_names = ['V1', 'V2', 'V3', 'V3A', 'V3B', 'V4', 'Laterial occipital']
for idx,name in zip(unique_roi_index,unique_roi_names):
    print(name,np.sum(dat['roi'] == idx))

V1 1294
V2 2083
V3 1790
V3A 484
V3B 314
V4 1535
Laterial occipital 928


In [5]:
#?make_pipeline()

In [6]:
def decoding_pipeline(BOLD_signal_voxels,manual_labels,
                      n_splits = 100,
                      ):
    feature_scaler = StandardScaler()
    model = linear_model.LogisticRegression(class_weight = 'balanced',random_state = 12345)
    pipeline = make_pipeline(feature_scaler,
                             model) #什么意思？ # 这是所谓的pipeline，在一个pipeline里有步骤，第一步对数据进行归中，第二部才拟合一个logisticregression
    cv = StratifiedShuffleSplit(n_splits = n_splits,test_size = 0.2, random_state = 12345)
    decoding_results = cross_validate(pipeline,
                                      BOLD_signal_voxels,
                                      manual_labels,
                                      groups = None,
                                      scoring = 'roc_auc',#什么意思？ # https://en.wikipedia.org/wiki/Receiver_operating_characteristic
                                      cv = cv,
                                      n_jobs = -1,
                                      return_estimator = True,
                                      verbose = 1,
                                      )
    return decoding_results#得到的是什么？ 得到一dictionary的变量，里面包含拟合的方程和每一次cross-validation的测试结果

In [7]:
results = dict()
for roi_name,idx_voxel in zip(unique_roi_names,unique_roi_index):
    voxel_selected = dat['responses'][:manual_labels.shape[0],dat['roi'] == idx_voxel]
    results[roi_name] = decoding_pipeline(voxel_selected,manual_labels,n_splits = 100)

ValueError: ignored

In [ ]:
df_plot = dict(roi_name = [],
               roc_auc = [],
               )
for roi_name,res in results.items():
    for score in res['test_score']:
        df_plot['roi_name'].append(roi_name)
        df_plot['roc_auc'].append(score)
df_plot = pd.DataFrame(df_plot)

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))
ax = sns.barplot(x = 'roi_name',
                 y = 'roc_auc',
                 data = df_plot,
                 ax = ax,
                 )
ax.axhline(0.5, linestyle = '--',color = 'black',label = 'chance level')
ax.set(ylim = (0.4,0.6))
ax.legend(loc = 'upper left')

# representational similarity analysis

In [ ]:
from scipy.spatial import distance

In [ ]:
fig,axes = plt.subplots(figsize = (20,20,),
                        nrows = 3,
                        ncols = 3,
                        )
for roi_name,idx_voxel,ax in zip(unique_roi_names,unique_roi_index,axes.flatten()):
    voxel_selected = dat['responses'][:manual_labels.shape[0],dat['roi'] == idx_voxel]
    idx_sort = np.argsort(manual_labels)
    X = voxel_selected[idx_sort]
    y = manual_labels[idx_sort]
    RDM = distance.squareform(distance.pdist(X,metric = 'correlation'))
    im = ax.imshow(RDM,cmap = plt.cm.coolwarm,vmin = .75,vmax = 1.5)
    ax.set(title = roi_name)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)